# Testing PostgreSQL Arrays

In [ ]:
import psycopg2 as pg
import pandas as pd
%matplotlib notebook

In [ ]:
connection = pg.connect("dbname=jeff user=jeff")

## Display Activities
Create a dataframe with one row per activity.

In [ ]:
query = """
SELECT id, name, timestamp, meta->'direction' AS direction,
meta->'robot' AS robot, meta->'wheel' AS wheel
FROM tc_activity
"""

activity = pd.read_sql(query, index_col='id', con=connection)
activity

## Display Activity Data
Select and pivot summary data for an activity.

In [ ]:
query = """
SELECT id, name, timestamp, measure, value
FROM tc_activity, unnest(activity_measures, data) AS u(measure, value);
"""

activity_data = pd.read_sql(query, con=connection)
activity_data

In [ ]:
pivot = activity_data.pivot(index='id', columns='measure', values='value')
pd.merge(activity, pivot, on='id')

## Trace Data
Select and pivot the trace data for most recent activity.

In [ ]:
query = """
SELECT t.millis, u.*
FROM tc_activity a, tc_trace t, unnest(a.trace_measures, t.data) AS u(measure, value)
WHERE a.id = t.activity_id AND a.timestamp = (SELECT max(timestamp) FROM tc_activity);
"""

trace_df = pd.read_sql(query, con=connection)
trace_df

In [ ]:
trace_df.pivot(index='millis', columns='measure', values='value')

## Trace Data
Select and pivot the trace data for activity with specific id.

In [ ]:
id = 9

query = """
SELECT a.id, t.millis, u.*
FROM tc_activity a, tc_trace t, unnest(a.trace_measures, t.data) AS u(measure, value)
WHERE a.id = t.activity_id AND a.id = %s;
"""

trace_df = pd.read_sql(query, con=connection, params=[id])
trace_df.pivot(index='millis', columns='measure', values='value')